In [11]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["HF_API_KEY"]=os.getenv("HF_API_KEY")


In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [7]:
#Builtin Tools:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [19]:
wikiAPI = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxviAPI = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)

arxiv = ArxivQueryRun(api_wrapper=arxviAPI)
wikipedia =WikipediaQueryRun(api_wrapper=wikiAPI)

In [22]:
# Custom tools: RAG tool
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma


In [23]:
loader = WebBaseLoader(web_path="https://oig.hhs.gov/oig-policies/official-site/")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap=100)
splitted_docs = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")
db = Chroma.from_documents(embedding=embeddings,documents=splitted_docs)

In [17]:
retriver = db.as_retriever()

In [24]:
from langchain.tools.retriever import create_retriever_tool

retriver_tool = create_retriever_tool(retriver,"RAG tool","Description: Searching info From US Health department")

In [25]:
# Creating chains of tools:

tools = [arxiv,wikipedia,retriver_tool]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt =ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful AI agent now solve users query with given tools"),
        (MessagesPlaceholder("chat_history")),
        ("user,","{input_Text}")
    ]
)

ValueError: Unexpected message type: user,. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.